# 서포트 벡터 머신

- 선형/비선형 분류, 회귀, 이상치 탐색 등에 사용할 수 있는 다목적 머신러닝 모델  
- 분류 문제나 작거나 중간 크기의 데이터셋에 적합  

## 5.1 선형 SVM 분류

서포트 벡터: 도로 경계에 위치한 샘플  

### 5.1.1 소프트 마진 분류

하드 마진 분류: 모든 샘플이 도로 바깥쪽에 올바르게 분류  
- 데이터가 선형적으로 구분될 수 있어야 제대로 작동  
- 이상치에 민감  

-> 좀 더 유연한 모델의 필요성!

소프트 마진 분류: 도로의 폭을 가능한 넓게 유지하는 것과 마진 오류 사이의 적절한 균형  
- 규제 파라미터 C: C값을 이용해 마진 조절 (C가 커지면 마진이 좁아지고 C가 작아지면 마진이 늘어남)  
- 힌지 손실: max(0, 1-t), t>=1 일때 0 (미분 불가능한 t=1에서 **서브그래디언트**를 사용해 경사하강법을 사용)   

In [6]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris["data"][:, (2,3)]
y = (iris["target"] == 2).astype(np.float64)

svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_svc", LinearSVC(C=1, loss="hinge")),
])
svm_clf.fit(X,y)

/Users/yeseo/miniconda/envs/ame/lib/python3.11/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Pipeline(steps=[('scaler', StandardScaler()),
                ('linear_svc', LinearSVC(C=1, loss='hinge'))])

In [3]:
svm_clf.predict([[5.5,1.7]])

array([1.])

## 5.2 비선형 SVM 분류

선형적으로 분류할 수 없는 데이터셋이 많음  
1. 다항 특성과 같은 특성을 더 추가하여 선형 분리
2. **커널 트릭**을 사용하여 분리  

### 5.2.1 다항식 커널

커널 트릭: 실제로 특성을 추가하지 않으면서 다항식 특성을 많이 추가한 것과 같은 결과  

In [4]:
from sklearn.svm import SVC
poly_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
])
poly_kernel_svm_clf.fit(X,y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=5, coef0=1, kernel='poly'))])

### 5.2.2 유사도 특성 추가

- 유사도 함수: 각 샘플이 특정 랜드마크와 얼마다 닮았는지 측정  (e.g. 가우시안 방사 기저 함수(**RBF**))  
- 랜드마크 선택 (e.g. 데이터 셋 모든 샘플 위치에 랜드마크 설정)
    - 차원이 매우 커지고 변환된 훈련 세트가 선형적으로 구분될 가능성이 높음  
    - 훈련 세트에 있는 n개의 특성을 가진 m개의 샘플이 m개의 특성을 가진 m개의 샘플로 변환됨 (훈련 세트가 매우 클 경우 동일한 크기의 아주 많은 특성이 만들어짐)

### 5.2.3 가우시안 RBF 커널

In [5]:
rbf_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
])
rbf_kernel_svm_clf.fit(X,y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=0.001, gamma=5))])

- gamma를 증가시키면 종 모양 그래프가 좁아지고 각 샘플의 영향 범위가 작아짐, 결정 경계가 더 불규칙해지고 각 샘플을 따라 구불구불하게 휘어짐  
- 작은 gamma 값은 넓은 종 모양을 만들며 샘플이 넒은 범위에 걸처 영향을 줌, 결정 경계가 부드러워짐  
  
결국 하이퍼파라미터 r가 규제의 역할을 수행 (과대적합일 경우 감소 / 과소적합일 경우 증가, 하이퍼파라미터 C와 비슷)  

다른 커널들(잘 사용되지는 않음)  
- 문자열 커널  
- 문자열 서브시퀀스 커널  
- 레벤슈타인 거리  

### 계산 복잡도

- LinearSVC: O(m * n)  
(m: 훈련 샘플 수, n: 특성 수)  
- SVC with 커널 트릭 알고리즘: O(m^2 * n) ~ O(m^3 * n)  
  - 복잡하지만 작거나 중간 규모의 훈련 세트에 잘 맞음  
  - **희소 특성**(각 샘플에 0이 아닌 특성이 몇개 없는 경우)인 경우 잘 확장됨  

## 5.3 SVM 회귀

- 분류: 일정한 마진 오류 안에서 두 클래스 간의 도로 폭이 가능한 한 최대가 되도록  
- 회귀: 제한된 마진 오류 안에서 도로 안에 가능한 한 많은 샘플이 들어가도록 학습  

마진 안에서는 훈련 샘플이 추가되어도 모델의 예측에는 영향이 없음(= $\epsilon$ insensitive model)

In [8]:
from sklearn.svm import SVR

svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1)
svm_poly_reg.fit(X,y)

SVR(C=100, degree=2, kernel='poly')

## 5.4 SVM 이론

### 5.4.1 결정 함수와 예측

선형 SVM 분류기를 훈련한다는 것은 가능한 한 마진을 크게 하는 w와 b를 찾는 것  
- 0 if w^T * X + b < 0  
- 1 if w^T * X + b >= 0

### 5.4.2 목적 함수

결정 함수의 기울기: 가중치 벡터 ||w||의 노름과 같음  
가중치 벡터 w가 작을수록 마진은 커짐  
- 하드 마진 선형 SVM 분류기의 목적함수를 제약이 있는 최적화 문제로 표현  
- **슬랙 변수**를 도입하여 소프트 마진 분류기의 목적함수를 구성

$minimize \frac{1}{2} w^t \cdot w + C\sum_{i=1}^{m}\xi^(i) $  

[조건] i=1,2,...m일 때 $t^(i)(w^T \cdot x^(i) + b) >= 1 -\xi^(i)$ 이고 $\xi^(i) >=0$    

### 5.4.3 콰드라틱 프로그래밍

콰드라틱 프로그래밍(QP): 선형적인 제약 조건이 있는 볼록 함수의 이차 최적화 문제  

### 5.4.4 쌍대 문제

**원 문제**라는 제약이 이쓴 최적화 문제가 주어지면 **쌍대 문제**라고 하는 깊게 관련된 다른 문제로 표현할 수 있음  
일반적으로 쌍대 문제 해는 원 문제 해의 하한값이지만, 어떤 조건 하에서는 원 문제와 똑같은 해를 제공함   

- SVM 문제는 이 조건을 만족시키기 때문에 원 문제 또는 쌍대문제를 선택하여 풀 수 있음
- 훈련 샘플 수가 특성 개수보다 작을 때, 원 문제보다 쌍대 문제를 푸는 것이 더 빠름
- 원 문제에서는 적용이 안 되는 커널 트릭을 가능하게 함  

### 5.4.5 커널 SVM

커널 트릭: 모든 훈련 샘플에 변환을 적용하는 것 대신 변환된 백터의 점곱을 간단하게 정의된 식으로 바꾸어 쓸 수 있는 효율적인 계산 기법  

#### 머서의 정리
머서의 정리에 따르면 함수 K(a,b)가 머서의 조건(K가 매개변수에 대해 연속, 대칭임. 즉 K(a,b)=K(b,a) 등)이라 부르는 몇 개 수학적 조건을 만족할 때 a와 b를 (더 높은 차원의) 다른 공간에 매핑하는 $K(a,b) = \phi(a)^T \cdot \phi(b)$와 같은 함수 $\phi$가 존재  

- 따라서 $\phi$를 모르더라도 존재하는 것은 알기 때문에 K를 커널로 사용할 수 있음  
- 가우시안 RBF 커널의 경우 $\phi$는 각 훈련샘플을 무한 차원의 공간에 매핑하는 것으로 볼 수 있음  

### 5.4.6 온라인 SVM

온라인 학습: 새로운 샘플이 생겼을 때 점진적으로 학습하는 것  

- 경사하강법 사용: QP 기반의 방법보다 훨씬 느리게 수렴  
- 온라인 커널 SVM 구현도 가능 (Incremental and Decremental SVM Learning, Fast Kernel Classifiers with Online and Active Learning)  

## 5.5 연습문제

### 1. 서포트 벡터 머신의 근본 아이디어는 무엇인가요?

### 2. 서포트 벡터가 무엇인가요?

### 3. SVM을 사용할 때 입력값의 스케일이 왜 중요한가요?

### 4. SVM 분류기가 샘플을 분류할 때 신뢰도 점수와 확률을 출력할 수 있나요?

### 5. 수백만 개의 샘플과 수백 개의 특성을 가진 훈련 세트에 SVM 모델을 훈련시키려면 원 문제와 쌍대 문제 중 어떤 것을 사용해야 하나요?

### 6. RBF 커널을 사용해 SVM 분류기를 훈련시켰더니 훈련 세트에 과소적합된 것 같습니다. r(gamma)를 증가시켜야 할까요, 감소시켜야 할까요? C의 경우는 어떤가요?

### 7. 이미 만들어진 QP 알고리즘 라이브러리를 사용해 소프트 마진 선형 SVM 분류기를 학습시키려면 QP 매개변수(H, f, A, b)를 어떻게 지정해야 하나요?

### 8. 선형적으로 분리되는 데이터셋에 LinearSVC를 훈련시켜보세요. 그런 다음 같은 데이터셋에 SVC와 SGDClassifier를 적용해보세요. 거의 비슷한 모델이 만들어지는지 확인해보세요.

### 9. MNIST 데이터셋에 SVM 분류기를 훈련시켜보세요. SVM 분류기는 이진 분류기라서 OvA 전략을 사용해 10개의 숫자를 분류해야 합니다. 처리 속도를 높이기 위해 작은 검증 세트로 하이퍼파라미터를 조정하는 것이 좋습니다. 어느 정도까지 정확도를 높일 수 있나요?

### 10. 캘리포니아 주택 가격 데이터셋에 SVM 회귀를 훈련시켜보세요.